In [1]:
# !pip install git+https://github.com/jupyterhub/binderhub
# !pip install https://github.com/jupyter/repo2docker/archive/master.zip

# ANGULAR SIZE VERSUS Z

In HU, the distance is given by
$d(z) = R_0 \frac{z}{(1+z)}$

the 4D radius is given by:
$R(z) = R_0 \frac{1}{(1+z)}$

where $R_0$ = 14.01 GLY
$G(t) = G_0\frac{R_0}{R(z)}$

$G = G_0 (1+z)$

Using Newtonian Dynamics while reversing time (having the average galaxy moving back in time and through the 4D spatial manifold), we get:
$G(z)\frac{M}{r(z)^2} = \frac{v(z)^2}{r(z)}$

r(z)=$G(z)\frac{M}{v(z)^2}=G_0 (1+z)\frac{M}{v(z)^2}$

Since the angular size of a galaxy is roughly r(z)/d(z)

AngularSize= $G_0 (1+z)\frac{M}{v(z)^2}/(R_0\frac{z}{(1+z)})= \frac{G_0M(1+z)^2}{R_0  z v(z)^2} \propto \frac{1}{z}$

This means that v(z) =$ v_0(1+z)$ which implies that $r(z) = r_0/(1+z)$

where $R_0$ is the current 4D radius (14.01 GLY), and $v_0$ is the current tangential velocity of a body in orbit.

So, as G varies, the radius of the orbits varies, and velocity adjusts to keep angular momentum constant. The resulting angular size z-dependency is $\frac{1}{z} $as depicted in Kapahi’s data.

**NOTICE THAT THIS IS NOT A SMALL RESULT. I DERIVED THE Z-DEPENDENCY OF THE ANGULAR SIZE (AN OBSERVABLE) IN AN EPOCH-DEPENDENT G. THIS HAS NEVER BEEN DONE, AND IT SUPPORTS MY THEORY AND DEBUNKS GENERAL RELATIVITY.**

**JWST'S OBSERVATION OF HUGE GALAXIES IS THE RESULT OF INCORRECT GALAXY SIZE PREDICTIONS AND DISTRIBUTION DUE TO RIEMMANIAN GEOMETRY: NO IFS OR BUTS.**


In [1]:
import math
import numpy as np
import xarray as xr
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


suffix = "_8"
pd_r = np.load("./data/_all_r" + suffix + ".npy")
pd_z = np.load("./data/_all_z" + suffix + ".npy")
pd_dv0 = np.load("./data/_all_dv0" + suffix + ".npy")
redshifts = np.load("./data/_all_redshifts" + suffix + ".npy")
epochs = np.load("./data/_epochs" + suffix + ".npy")
n_epochs = len(epochs)
Radius_4D=14.01e9


# Load the dataset
data = np.load ("./data/_all_current_masses" + suffix + ".npy")
pd_masses = xr.DataArray(data).rename({ 'dim_0' : 'epoch', 'dim_1': 'radius', 'dim_2': 'elevation',})

def plot_2d_mass(z):
    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    
    R_0 = 14.01  # 4D radius of the Universe
    z_0 = redshifts[z]  # the initial redshift
    
    # Create the 2D subplot
    ax = plt.subplot(gs[0])
    

    # Create a symmetric X grid by concatenating the negative and positive r values
    pd_r[z][0] = 0.0
    r_extended = np.concatenate((-pd_r[z][::-1], pd_r[z]))

    # Concatenate the values of pd_masses to make the mass symmetric along the radius
    mass_values_symmetric = np.concatenate((pd_masses[z].values[::-1, :], pd_masses[z].values), axis=0)

    # Check the shapes to understand the mismatch
    print("Shape of mass_values_symmetric:", mass_values_symmetric.shape)
    print("Shape of pd_z[z]:", pd_z[z].shape)
    print("Shape of r_extended:", r_extended.shape)

    # Create a new DataArray with symmetric values
    symmetric_mass = xr.DataArray(
        mass_values_symmetric, 
        coords={'radius': r_extended, 'elevation': pd_z[z]}, 
        dims=['radius', 'elevation']  # swap these dimensions
    )



    dv0 = pd_dv0[z][:, None]
    dv0_broadcasted = np.broadcast_to(dv0, pd_masses[z].values.shape)

    # Now, you can plot this symmetric_mass DataArray directly
    symmetric_mass.plot(ax=ax, x='radius', y='elevation', cmap='coolwarm')
    ax.set_title("$M_{33}$ Mass Profile\n Redshift z = %.2f" % z_0)
    ax.set_xlabel('X (1000 Lyr)')
    ax.set_ylabel('Z (1000 Lyr)')

    # Remaining polar subplot code remains unchanged
    # Create the second subplot
    ax = plt.subplot(gs[1], projection='polar')  # 1 row, 2 columns, second subplot
    circle_outer = plt.Circle((0, 0), R_0, transform=ax.transData._b, fill = False)
    circle_inner = plt.Circle((0, 0), R_0/(1+z_0), transform=ax.transData._b, fill = False, linestyle='dashed')
    ax.add_artist(circle_outer)
    ax.add_artist(circle_inner)

    z_values = np.linspace(0, z_0, 100)
    R_values = R_0/(1+z_values)
    R_z0 =  R_0/(1+z_0)
    angle_0 = 1 - 1/(1+z_0)
    angle_values =  R_values/R_0
    ax.plot(np.pi/2-angle_0, R_z0, 'ro')
    ax.text(np.pi/2, R_0, 'Earth', horizontalalignment='right')
    ax.text((np.pi/2-angle_0)*0.92, R_z0, '$M_{33}$', horizontalalignment='left')
    ax.scatter(np.pi/2 - angle_0 + (R_values-R_z0)/R_0, R_values, color='b', s=0.5)
    ax.set_rmax(R_0)
    ax.set_rticks([])  # Less radial ticks
    ax.set_rlabel_position(-24.5)  # Move radial labels away from plotted line
    ax.grid(True)

    ax.set_title("$M_{33}$ Ancient Photon's path across \nthe Hyperspherical Universe", va='bottom')

    plt.subplots_adjust(wspace=0.3)
    plt.show()

z_slider = widgets.IntSlider(min=0, max=n_epochs-1, step=1, value=0, description='Redshift:', continuous_update=False)
interactive_plot = widgets.interactive_output(plot_2d_mass, {'z': z_slider})
display(z_slider, interactive_plot)


IntSlider(value=0, continuous_update=False, description='Redshift:', max=19)

Output()

In [2]:
def plot_2d_mass(z):
    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    
    R_0 = 14.01  # 4D radius of the Universe
    z_0 = redshifts[z]  # the initial redshift
    
    # Create the 2D subplot
    ax = plt.subplot(gs[0])

    # Create a symmetric X grid by concatenating the negative and positive r values
    pd_r[z][0]=1
    r_extended = np.concatenate((-pd_r[z][::-1], pd_r[z]))
    
    # Concatenate the values of pd_masses to make the mass symmetric along the radius
    mass_values_symmetric = np.concatenate((pd_masses[z].values[::-1, :], pd_masses[z].values), axis=0)

    # Calculate the density by dividing mass values by dv0
    dv0 = pd_dv0[z][:, None]
    r_exte4nded_squared = r_extended*r_extended
    r_exte4nded_squared = r_exte4nded_squared[:, None]
#     density_values_symmetric = np.log( mass_values_symmetric / r_exte4nded_squared ) #np.concatenate((dv0[::-1], dv0))
    density_values_symmetric = np.log( mass_values_symmetric / np.concatenate((dv0[::-1], dv0)) ) 

    
    # Create a new DataArray for symmetric density
    symmetric_density = xr.DataArray(
        density_values_symmetric, 
        coords={'radius': r_extended, 'elevation': pd_z[z]}, 
        dims=['radius', 'elevation']
    )

    # Now, you can plot this symmetric_density DataArray directly
    symmetric_density.plot(ax=ax, x='radius', y='elevation', cmap='coolwarm')
    ax.set_title("$M_{33}$ Density Profile\n Redshift z = %.2f" % z_0)
    ax.set_xlabel('X (Lyr)')
    ax.set_ylabel('Z (Lyr)')

    # Remaining polar subplot code remains unchanged
     # Create the second subplot
    ax = plt.subplot(gs[1], projection='polar')  # 1 row, 2 columns, second subplot
    circle_outer = plt.Circle((0, 0), R_0, transform=ax.transData._b, fill = False)
    circle_inner = plt.Circle((0, 0), R_0/(1+z_0), transform=ax.transData._b, fill = False, linestyle='dashed')
    ax.add_artist(circle_outer)
    ax.add_artist(circle_inner)

    z_values = np.linspace(0, z_0, 100)
    R_values = R_0/(1+z_values)
    R_z0 =  R_0/(1+z_0)
    angle_0 = 1 - 1/(1+z_0)
    angle_values =  R_values/R_0
    ax.plot(np.pi/2-angle_0, R_z0, 'ro')
    ax.text(np.pi/2, R_0, 'Earth', horizontalalignment='right')
    ax.text((np.pi/2-angle_0)*0.92, R_z0, '$M_{33}$', horizontalalignment='left')
    ax.scatter(np.pi/2 - angle_0 + (R_values-R_z0)/R_0, R_values, color='b', s=0.5)
    ax.set_rmax(R_0)
    ax.set_rticks([])  # Less radial ticks
    ax.set_rlabel_position(-24.5)  # Move radial labels away from plotted line
    ax.grid(True)

    ax.set_title("$M_{33}$ Ancient Photon's path across \nthe Hyperspherical Universe", va='bottom')
    plt.subplots_adjust(wspace=0.3)
    plt.show()

z_slider = widgets.IntSlider(min=0, max=n_epochs-1, step=1, value=0, description='Redshift:', continuous_update=False)
interactive_plot = widgets.interactive_output(plot_2d_mass, {'z': z_slider})
display(z_slider, interactive_plot)


IntSlider(value=0, continuous_update=False, description='Redshift:', max=19)

Output()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import xarray as xr
import ipywidgets as widgets
from ipywidgets import interactive

R_0 = 14.01E9
suffix ="_10"
pd_r = np.load("./data/_all_r" + suffix + ".npy")
pd_z = np.load("./data/_all_z" + suffix + ".npy")
pd_dv0 = np.load("./data/_all_dv0" + suffix + ".npy")
redshifts = np.load("./data/_all_redshifts" + suffix + ".npy")
epochs = np.load("./data/_epochs" + suffix + ".npy")
n_epochs = len(epochs)
pd_masses = xr.DataArray(np.load("./data/_all_current_masses" + suffix + ".npy")).rename({'dim_0' : 'epoch', 'dim_1': 'radius', 'dim_2': 'elevation'})


def update_density_range(z):
    min_density = np.log( np.min(pd_masses[z].values / pd_dv0[z][:, None]) +1E-5 )
    max_density = np.log( np.max(pd_masses[z].values / pd_dv0[z][:, None]) )
    if (min_density > max_density):
        a = max_density
        max_density = min_density
        min_density = a
    density_range_slider.max = max_density
    density_range_slider.min = min_density
    density_range_slider.value = [min_density, max_density]
    density_range_slider.step = (max_density - min_density) / 100

def plot_2d_mass(z, density_range):
    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    
    R_0 = 14.01  # 4D radius of the Universe
    z_0 = redshifts[z]  # the initial redshift
    
    # Create the 2D subplot
    ax = plt.subplot(gs[0])

    # Create a symmetric X grid by concatenating the negative and positive r values
    pd_r[z][0]=0.0
    r_extended = np.concatenate((-pd_r[z][::-1], pd_r[z]))

    # Concatenate the values of pd_masses to make the mass symmetric along the radius
    mass_values_symmetric = np.concatenate((pd_masses[z].values[::-1, :], pd_masses[z].values), axis=0)

    # Calculate the density by dividing mass values by dv0
    dv0 = pd_dv0[z][:, None]
    density_values_symmetric = mass_values_symmetric / np.concatenate((dv0[::-1], dv0))

    # Create a new DataArray for symmetric density
    symmetric_density = xr.DataArray(
        density_values_symmetric, 
        coords={'radius': r_extended, 'elevation': pd_z[z]}, 
        dims=['radius', 'elevation']
    )

    # Now, you can plot this symmetric_density DataArray directly
    symmetric_density.plot(ax=ax, x='radius', y='elevation', cmap='coolwarm', vmin=density_range[0], vmax=density_range[1])
    ax.set_title("$M_{33}$ Luminous Density Profile\n Redshift z = %.2f" % z_0)
    ax.set_xlabel('X (1000 Lyr)')
    ax.set_ylabel('Z (1000 Lyr)')

    # Remaining polar subplot code remains unchanged
    # Create the second subplot
    ax = plt.subplot(gs[1], projection='polar')  # 1 row, 2 columns, second subplot
    circle_outer = plt.Circle((0, 0), R_0, transform=ax.transData._b, fill = False)
    circle_inner = plt.Circle((0, 0), R_0/(1+z_0), transform=ax.transData._b, fill = False, linestyle='dashed')
    ax.add_artist(circle_outer)
    ax.add_artist(circle_inner)

    z_values = np.linspace(0, z_0, 100)
    R_values = R_0/(1+z_values)
    R_z0 =  R_0/(1+z_0)
    angle_0 = 1 - 1/(1+z_0)
    angle_values =  R_values/R_0
    ax.plot(np.pi/2-angle_0, R_z0, 'ro')
    ax.text(np.pi/2, R_0, 'Earth', horizontalalignment='right')
    ax.text((np.pi/2-angle_0)*0.92, R_z0, '$M_{33}$', horizontalalignment='left')
    ax.scatter(np.pi/2 - angle_0 + (R_values-R_z0)/R_0, R_values, color='b', s=0.5)
    ax.set_rmax(R_0)
    ax.set_rticks([])  # Less radial ticks
    ax.set_rlabel_position(-24.5)  # Move radial labels away from plotted line
    ax.grid(True)

    ax.set_title("$M_{33}$ Ancient Photon's path across \nthe Hyperspherical Universe", va='bottom')

    plt.subplots_adjust(wspace=0.3)
    plt.show()



# Slider for redshift
z_slider = widgets.IntSlider(min=0, max=n_epochs-1, step=1, value=0, description='Redshift:', continuous_update=False)

# Slider for density range (initialize with dummy values; they'll be updated by the function)
density_range_slider = widgets.FloatRangeSlider(
    value=[0, 1],
    min=0,
    max=1,
    step=0.01,
    description='Density Range:',
    continuous_update=False
)

def on_z_slider_change(change):
    if change['name'] == 'value':
        update_density_range(change['new'])

z_slider.observe(on_z_slider_change)

interactive_plot = widgets.interactive_output(plot_2d_mass, {'z': z_slider, 'density_range': density_range_slider})
display(z_slider, density_range_slider, interactive_plot)

# Trigger the initial update
update_density_range(z_slider.value)

IntSlider(value=0, continuous_update=False, description='Redshift:', max=19)

FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='Density Range:', max=1.0, step=0.01)

Output()